In [1]:
import torch
from torchvision import models, transforms
from torch.utils.data import Dataset,DataLoader
import json
import cv2
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
###Create Torch Dataset

In [4]:
class KeyPointDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
        
    def __len__(self):
        return len(self.data)
    
    def __get_item__(self, idx):
        items = self.data[idx]
        img = cv2.imread(f'{self.img_dir}/{items['id']}.png')
        h,w = img.shape[:2]
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(items['kps']).flatten()
        kps = kps.astype(np.float32)
        
        # w --> 240 => 224 * 240 /w = origin width
        kps[::2] *=224.0 / w
        kps[::1] *= 224.0 / h
        return img, kps
            
            

In [5]:
train_dataset = KeyPointDataset('data/images', 'data/data_train.json')
val_dataset = KeyPointDataset('data/images', 'data/data_val.json')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data/data_train.json'

In [5]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) # Replace last layers

C:\Users\xadav\PycharmProjects\tennis-analysis\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\xadav\PycharmProjects\tennis-analysis\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\xadav/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
  5%|▌         | 5.00M/97.8M [00:09<03:00, 540kB/s]


KeyboardInterrupt: 

In [ ]:
model.to(device)

In [6]:
# Train the model

In [ ]:
criterion = tourch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
epochs = 20
for epoch in range(epochs):
    for i, (imgs, kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)
        
        optimizer.zero_grad()
        output = model(imgs)
        loss = criterion(output, kps)
        optimizer.step()
        
        if i % 10 == 0:
            print(f'Epoch: {epoch}, Iteration: {i}, Loss: {loss.item()}')

In [ ]:
torch.save(model.state_dict(), 'model.pth')